## Hedging just asset 1 

### With Transaction Costs

In [ ]:
reset()# Clears all the parameters
import numpy as np
import pandas as pd
from scipy.stats import norm
import math
from sage.misc.table import table
import matplotlib.pyplot as plt
from scipy.integrate import quad

def black_scholes_calculator(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    black_scholes_call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return black_scholes_call_price

def delta_calculator(S, K, r, tau, sigma):
    if tau > 0:
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        return norm.cdf(d1)
    else:
        return 1 if S >= K else 0

def stock_price_simulator(S0_1, S0_2, mu_1, mu_2, sigma_1, sigma_2, T, num_steps,n, rho):
    dt = T / n
    stock_path_1 = [S0_1]
    stock_path_2 = [S0_2]
    
    for _ in range(1, n + 1):  
        Z1 = np.random.normal()
        Z2 = rho * Z1 + np.sqrt(1 - rho**2) * np.random.normal()

        S1_new = stock_path_1[-1] * np.exp((mu_1 - 0.5 * sigma_1**2) * dt + sigma_1 * np.sqrt(dt) * Z1)
        S2_new = stock_path_2[-1] * np.exp((mu_2 - 0.5 * sigma_2**2) * dt + sigma_2 * np.sqrt(dt) * Z2)

        stock_path_1.append(S1_new)
        stock_path_2.append(S2_new)

    return np.array(stock_path_1), np.array(stock_path_2)

def Unhedged_value(St, K, mu, sigma, T):
    zt = -1/(sigma* sqrt(T))*((np.log(St/K) + (mu - 0.5 * sigma**2) * T))
    
    def integrand_St(zt):
        part1 = np.exp((mu - 0.5 * sigma**2) * T + sigma*sqrt(T)*zt)
        part2 = np.exp(-0.5*(zt**2)) 
        return (part1 * part2) / np.sqrt(2 * np.pi )
    
    def integrand_K(zt):
        return np.exp(-0.5*(zt**2)) / np.sqrt(2 * np.pi)
    
    integral_St, _ = quad(integrand_St, zt, np.inf)
    integral_K, _ = quad(integrand_K, zt, np.inf)
    
    C = np.exp(-r * T) * (St*integral_St - K * integral_K)
    
    return C

def rebalancing_days(n,num_steps):
    return [round(i*n//num_steps) for i in range(1,num_steps+1)]

def portfolio_calculator(S0_1, S0_2, K, r, T, sigma_1, sigma_2, num_steps,n, mu_1, mu_2, rho, alpha):
    stock_path_1, stock_path_2 = stock_price_simulator(S0_1, S0_2, mu_1, mu_2, sigma_1, sigma_2, T, num_steps,n, rho)
    sigma=sqrt((alpha**2)*(sigma_1**2) +  ((1-alpha)**2)*(sigma_2**2)  +  2 * rho*(alpha)*(1-alpha) * sigma_1 * sigma_2)
    dt = T/n
    
    rebalancing_day=rebalancing_days(n,num_steps)
    
    initial_portfolio_price_St = (alpha*S0_1+(1-alpha)*S0_2)
    
    black_scholes_callprice = black_scholes_calculator(initial_portfolio_price_St, K, r, T, sigma)
    delta_initial = delta_calculator(S0_1, K, r, T, sigma_1)
    
    portfolio_value = S0_1 * delta_initial  * (1-k_1)
    
    tau=1

    """
    # Header for the table
    print(f"{'Period':.6}\t{'tau':.10}\t{'delta':<10}\t{'delta_before':.15}\t{'St1':.10}\t\t{'St2':.10}\t\t{'St':.10}\t{'Portfolio Value':.15}")
    print(f"{0:<6}\t{tau:<10}{round(delta_initial, 4):<10.4f}\t{'':<10}\t{round(S0_1, 2):<15.2f}{round(S0_2, 2):<10.2f}\t{round(initial_portfolio_price_St, 2):<10.2f}\t{round(portfolio_value, 2):<20.2f}")
    """
    
    delta_before = delta_initial
    for t in range(1,n+ 1):

        St_1 = stock_path_1[t]
        St_2 = stock_path_2[t]
        tau = T - t * dt  


        if t in rebalancing_day:
            if tau >0:
                current_portfolio_price_St = St_1

                delta = delta_calculator(current_portfolio_price_St, K, r, tau, sigma_1)

                portfolio_value = (1 + r * dt) * portfolio_value + (delta - delta_before) * current_portfolio_price_St * (1 - np.sign(delta - delta_before) * k_1)
                """
                print(f"{t:<6}\t{round(tau, 2):<10.2f}{round(delta, 4):<10.4f}\t{round(delta_before, 4):<10.4f}\t{round(St_1, 2):<15.2f}{round(St_2, 2):<10.2f}\t{round(current_portfolio_price_St, 2):<10.2f}\t{round(portfolio_value, 2):<20.2f}")
                """
                delta_before = delta

            if tau==0:
            
                St_1_final = St_1   
                St_2_final = St_2   
                S_T = (alpha*St_1_final+(1-alpha)*St_2_final)

                delta_final = delta_calculator(St_1_final, K, r, 0, sigma_1)

                final_portfolio_value =(1 + r * dt) * portfolio_value + (delta_final - delta_before) * St_1_final * (1 - np.sign(delta_final - delta_before) * k_1)- (delta_final * St_1_final) + max(S_T - K, 0)
                simulatedprice = (np.exp(-r * T)) * final_portfolio_value

                C_T = max(S_T - K, 0)

                Measure = float((np.exp(-r * T)) * C_T)
                mu = (alpha*mu_1+(1-alpha)*mu_2)
                unhedged_value = Unhedged_value(initial_portfolio_price_St, K, mu, sigma,T)  

                """
                print(f"{t:<6}\t{round(tau,2):<10.2f}{round(delta_final, 4):<10.4f}\t{round(delta_before, 4):<10.4f}\t{round(St_1_final, 2):<15.2f}{round(St_2_final, 2):<11.2f}\t{round(S_T, 2):<10.2f}\t{round(final_portfolio_value, 2):<20.2f}")
                """

                delta_before = delta 

                return simulatedprice,S_T,Measure,unhedged_value


S0_1 = 50    
S0_2 = 50     

alpha_values = np.round(np.linspace(0, 1, 11), 1)  
rho = 0.2 
K = 50          
r = 0.02        
T = 1             
sigma_1 = 0.2      
sigma_2 = 0.2 

mu_1 = 0.02 
mu_2 = 0.02 
n=252
num_steps = 252 

k_1=0.005 
num_simulations = 10000

resultsTC_z2_i_hedging_asset_one_in_presence_of_costs=[]

trading_just_asset1=[]
for alpha in alpha_values:
    final_portfolio_prices_St=[]
    simulatedprices=[]
    avgsimulatedprices=[]
    avgfinal_portfolio_prices_St=[]
    std_measure_values =[]

    standard_deviation_of_the_simprices=[]
    standard_deviation_of_the_final_portfolio_prices_St=[]
    measure_values =[]
    discounted_P_measure_values=[]
    unhedged_values=[]
    Avg_unhedged_values=[]
    initial_portfolio_price_St=(alpha*S0_1+(1-alpha)*S0_2)
    sigma=sqrt((alpha**2)*(sigma_1**2) +  ((1-alpha)**2)*(sigma_2**2)  +  2 * rho*(alpha)*(1-alpha) * sigma_1 * sigma_2)
    black_scholes_callprice = black_scholes_calculator(initial_portfolio_price_St, K, r, T, sigma)

    for run in range(num_simulations):
        simulatedprice,S_T,Measure,unhedged_value =portfolio_calculator(S0_1, S0_2, K, r, T, sigma_1, sigma_2, num_steps,n, mu_1, mu_2, rho,alpha)
        simulatedprices.append(simulatedprice)
        final_portfolio_prices_St.append(S_T)
        measure_values.append(Measure)
        unhedged_values.append(unhedged_value)
        
        
        resultsTC_z2_i_hedging_asset_one_in_presence_of_costs.append((alpha,black_scholes_callprice,S_T,simulatedprice,Measure))

    sum_price=sum(simulatedprices)
    sum_finalstockprice=sum(final_portfolio_prices_St)
    
    squared_diffs = [(x - (sum_price/len(simulatedprices)) )** 2 for x in simulatedprices]
    varianceofprices = sum(squared_diffs) / len(simulatedprices)
    std_dev_simprices = math.sqrt(varianceofprices)
    standard_deviation_of_the_simprices.append(std_dev_simprices)
    avgsimulatedprice=np.sum(simulatedprices)/(len(simulatedprices))
    avgsimulatedprices.append(avgsimulatedprice) 
    
    squared_diff_finalstockprice = [(y - (sum_finalstockprice/len(final_portfolio_prices_St)) )** 2 for y in final_portfolio_prices_St]
    varianceofprices_finalstockprice = sum(squared_diff_finalstockprice) / len(final_portfolio_prices_St)
    std_dev_finalstockprice = math.sqrt(varianceofprices_finalstockprice)
    standard_deviation_of_the_final_portfolio_prices_St.append(std_dev_finalstockprice)
    
    avgS_T=np.sum(final_portfolio_prices_St)/(len(final_portfolio_prices_St))
    avgfinal_portfolio_prices_St.append(avgS_T) 
    
    squared_diffs_measure = [(x - sum(measure_values)/(len(measure_values)) )** 2 for x in measure_values]
    varianceofprices_measure = sum(squared_diffs_measure) / len(measure_values)
    std_dev_measure = math.sqrt(varianceofprices_measure)
        
    discounted_P_measure_value=np.sum(measure_values)/(len(measure_values))
    discounted_P_measure_values.append(discounted_P_measure_value)   
    
    Avg_unhedged_value=np.sum(unhedged_values)/(len(unhedged_values))
    Avg_unhedged_values.append(Avg_unhedged_value) 
    
    trading_just_asset1.append((alpha,black_scholes_callprice,avgsimulatedprice,std_dev_simprices,avgS_T,std_dev_finalstockprice,discounted_P_measure_value,std_dev_measure,Avg_unhedged_value))

df = pd.DataFrame(resultsTC_z2_i_hedging_asset_one_in_presence_of_costs, columns=['alpha', 'black_scholes_callprice', 'S_T', 'simulatedprice','Measure'])

df['alpha'] = df['alpha'].round(1)
df['black_scholes_callprice'] = df['black_scholes_callprice'].round(2)
df['S_T'] = df['S_T'].round(2)
df['simulatedprice'] = df['simulatedprice'].round(2)
df['Measure']=df['Measure'].round(2)

df.to_csv('resultsTC_z2_i_hedging_asset_one_in_presence_of_costs.csv', index=False, float_format='%.2f')
"""
print("Results have been written to resultsTC_z2_i_hedging_asset_one_in_presence_of_costs.csv")

# Just to view the contents in the csv file from here
with open('resultsTC_z2_i_hedging_asset_one_in_presence_of_costs.csv', 'r') as f:
    rows = [line.strip().split(',') for line in f]

# Create a table from the rows
table_output = table(rows)

# Print the table
print(table_output)
"""
df1 = pd.DataFrame(trading_just_asset1, columns=['alpha','black_scholes_price', 'avgsimulatedprice', 'std_dev_simprices','avgS_T','std_dev_finalstockprice','discounted_P_measure_value','std_dev_measure','Avg_unhedged_value'])

df1['alpha'] = df1['alpha'].round(2)
df1['black_scholes_price']=df1['black_scholes_price'].round(2)
df1['avgsimulatedprice'] = df1['avgsimulatedprice'].round(2)
df1['std_dev_simprices'] = df1['std_dev_simprices'].round(2)
df1['avgS_T'] = df1['avgS_T'].round(2)
df1['std_dev_finalstockprice'] = df1['std_dev_finalstockprice'].round(2)
df1['discounted_P_measure_value'] = df1['discounted_P_measure_value'].round(2)
df1['std_dev_measure'] = df1['std_dev_measure'].round(2)
df1['Avg_unhedged_value'] = df1['Avg_unhedged_value'].round(2)

df1.to_csv('trading_just_asset1.csv', index=False, float_format='%.2f')
"""
print("Results have been written to trading_just_asset1.csv")
"""
with open('trading_just_asset1.csv', 'r') as f:
    rows = [line.strip().split(',') for line in f]
new_headers = ["alpha", "Black S price", "avgsim.price", "stdsimprice", "avgS_T", "std_S_T", "discounted_P_val","std_measure" "Avg_unhedged_val"]
def make_first_letter_bold(text):
    return f"\033[1m{text[0]}\033[0m{text[1:]}"

bold_headers = [make_first_letter_bold(header) for header in new_headers]

fixed_width = 10.5 

formatted_header = ' '.join(f"{header:<{fixed_width}}" for header in bold_headers)

formatted_rows = [formatted_header]  
for row in rows[1:]:  
    formatted_row = ' '.join(f"{cell:<{fixed_width}}" for cell in row)
    formatted_rows.append(formatted_row)

table_output = '\n'.join(formatted_rows)

print(table_output)

df = pd.read_csv('trading_just_asset1.csv')  
plt.figure(figsize=(10, 6))
plt.plot(df['alpha'], df['black_scholes_price'],'-o', label='BS price')
plt.plot(df['alpha'], df['avgsimulatedprice'],'-.', label='avg sim price')
plt.plot(df['alpha'], df['discounted_P_measure_value'],'-*', label='dis P_measure')
plt.plot(df['alpha'], df['Avg_unhedged_value'], label='Avg_unhedged')

plt.xlabel('alpha values',fontsize =14)
plt.ylabel('Simulated Values',fontsize =14)
plt.legend()

plt.show()


## Without Transaction Costs

In [ ]:
reset()# Clears all the parameters
import numpy as np
import pandas as pd
from scipy.stats import norm
import math
from sage.misc.table import table
import matplotlib.pyplot as plt
from scipy.integrate import quad

def black_scholes_calculator(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    black_scholes_call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return black_scholes_call_price

def delta_calculator(S, K, r, tau, sigma):
    if tau > 0:
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        return norm.cdf(d1)
    else:
        return 1 if S >= K else 0

def stock_price_simulator(S0_1, S0_2, mu_1, mu_2, sigma_1, sigma_2, T, num_steps,n, rho):
    dt = T / n
    stock_path_1 = [S0_1]
    stock_path_2 = [S0_2]
    
    for _ in range(1, n + 1):  
        Z1 = np.random.normal()
        Z2 = rho * Z1 + np.sqrt(1 - rho**2) * np.random.normal()

        S1_new = stock_path_1[-1] * np.exp((mu_1 - 0.5 * sigma_1**2) * dt + sigma_1 * np.sqrt(dt) * Z1)
        S2_new = stock_path_2[-1] * np.exp((mu_2 - 0.5 * sigma_2**2) * dt + sigma_2 * np.sqrt(dt) * Z2)

        stock_path_1.append(S1_new)
        stock_path_2.append(S2_new)

    return np.array(stock_path_1), np.array(stock_path_2)

def Unhedged_value(St, K, mu, sigma, T):
    zt = -1/(sigma* sqrt(T))*((np.log(St/K) + (mu - 0.5 * sigma**2) * T))
    
    def integrand_St(zt):
        part1 = np.exp((mu - 0.5 * sigma**2) * T + sigma*sqrt(T)*zt)
        part2 = np.exp(-0.5*(zt**2)) 
        return (part1 * part2) / np.sqrt(2 * np.pi )
    
    def integrand_K(zt):
        return np.exp(-0.5*(zt**2)) / np.sqrt(2 * np.pi)
    
    integral_St, _ = quad(integrand_St, zt, np.inf)
    integral_K, _ = quad(integrand_K, zt, np.inf)
    
    C = np.exp(-r * T) * (St*integral_St - K * integral_K)
    
    return C

def rebalancing_days(n,num_steps):
    return [round(i*n//num_steps) for i in range(1,num_steps+1)]

def portfolio_calculator(S0_1, S0_2, K, r, T, sigma_1, sigma_2, num_steps,n, mu_1, mu_2, rho, alpha):
    stock_path_1, stock_path_2 = stock_price_simulator(S0_1, S0_2, mu_1, mu_2, sigma_1, sigma_2, T, num_steps,n, rho)
    sigma=sqrt((alpha**2)*(sigma_1**2) +  ((1-alpha)**2)*(sigma_2**2)  +  2 * rho*(alpha)*(1-alpha) * sigma_1 * sigma_2)
    dt = T/n
    
    rebalancing_day=rebalancing_days(n,num_steps)
    
    initial_portfolio_price_St = (alpha*S0_1+(1-alpha)*S0_2)
    
    black_scholes_callprice = black_scholes_calculator(initial_portfolio_price_St, K, r, T, sigma)
    delta_initial = delta_calculator(S0_1, K, r, T, sigma_1)
    
    portfolio_value = S0_1 * delta_initial  * (1-k_1)
    
    tau=1

    """
    # Header for the table
    print(f"{'Period':.6}\t{'tau':.10}\t{'delta':<10}\t{'delta_before':.15}\t{'St1':.10}\t\t{'St2':.10}\t\t{'St':.10}\t{'Portfolio Value':.15}")
    print(f"{0:<6}\t{tau:<10}{round(delta_initial, 4):<10.4f}\t{'':<10}\t{round(S0_1, 2):<15.2f}{round(S0_2, 2):<10.2f}\t{round(initial_portfolio_price_St, 2):<10.2f}\t{round(portfolio_value, 2):<20.2f}")
    """
    
    delta_before = delta_initial
    for t in range(1,n+ 1):

        St_1 = stock_path_1[t]
        St_2 = stock_path_2[t]
        tau = T - t * dt  


        if t in rebalancing_day:
            if tau >0:
                current_portfolio_price_St = St_1

                delta = delta_calculator(current_portfolio_price_St, K, r, tau, sigma_1)

                portfolio_value = (1 + r * dt) * portfolio_value + (delta - delta_before) * current_portfolio_price_St * (1 - np.sign(delta - delta_before) * k_1)
                """
                print(f"{t:<6}\t{round(tau, 2):<10.2f}{round(delta, 4):<10.4f}\t{round(delta_before, 4):<10.4f}\t{round(St_1, 2):<15.2f}{round(St_2, 2):<10.2f}\t{round(current_portfolio_price_St, 2):<10.2f}\t{round(portfolio_value, 2):<20.2f}")
                """
                delta_before = delta

            if tau==0:
            
                St_1_final = St_1   
                St_2_final = St_2   
                S_T = (alpha*St_1_final+(1-alpha)*St_2_final)

                delta_final = delta_calculator(St_1_final, K, r, 0, sigma_1)

                final_portfolio_value =(1 + r * dt) * portfolio_value + (delta_final - delta_before) * St_1_final * (1 - np.sign(delta_final - delta_before) * k_1)- (delta_final * St_1_final) + max(S_T - K, 0)
                simulatedprice = (np.exp(-r * T)) * final_portfolio_value

                C_T = max(S_T - K, 0)

                Measure = float((np.exp(-r * T)) * C_T)
                mu = (alpha*mu_1+(1-alpha)*mu_2)
                unhedged_value = Unhedged_value(initial_portfolio_price_St, K, mu, sigma,T)  

                """
                print(f"{t:<6}\t{round(tau,2):<10.2f}{round(delta_final, 4):<10.4f}\t{round(delta_before, 4):<10.4f}\t{round(St_1_final, 2):<15.2f}{round(St_2_final, 2):<11.2f}\t{round(S_T, 2):<10.2f}\t{round(final_portfolio_value, 2):<20.2f}")
                """

                delta_before = delta 

                return simulatedprice,S_T,Measure,unhedged_value


S0_1 = 50    
S0_2 = 50     

alpha_values = np.round(np.linspace(0, 1, 11), 1)  
rho = 0.2 
K = 50          
r = 0.02        
T = 1             
sigma_1 = 0.2      
sigma_2 = 0.2 

mu_1 = 0.02 
mu_2 = 0.02 
n=252
num_steps = 252 

k_1= 0.0
num_simulations = 10000

resultsTC_z2_i_hedging_asset_one_in_presence_of_costs=[]

trading_just_asset1=[]
for alpha in alpha_values:
    final_portfolio_prices_St=[]
    simulatedprices=[]
    avgsimulatedprices=[]
    avgfinal_portfolio_prices_St=[]
    std_measure_values =[]

    standard_deviation_of_the_simprices=[]
    standard_deviation_of_the_final_portfolio_prices_St=[]
    measure_values =[]
    discounted_P_measure_values=[]
    unhedged_values=[]
    Avg_unhedged_values=[]
    initial_portfolio_price_St=(alpha*S0_1+(1-alpha)*S0_2)
    sigma=sqrt((alpha**2)*(sigma_1**2) +  ((1-alpha)**2)*(sigma_2**2)  +  2 * rho*(alpha)*(1-alpha) * sigma_1 * sigma_2)
    black_scholes_callprice = black_scholes_calculator(initial_portfolio_price_St, K, r, T, sigma)

    for run in range(num_simulations):
        simulatedprice,S_T,Measure,unhedged_value =portfolio_calculator(S0_1, S0_2, K, r, T, sigma_1, sigma_2, num_steps,n, mu_1, mu_2, rho,alpha)
        simulatedprices.append(simulatedprice)
        final_portfolio_prices_St.append(S_T)
        measure_values.append(Measure)
        unhedged_values.append(unhedged_value)
        
        
        resultsTC_z2_i_hedging_asset_one_in_presence_of_costs.append((alpha,black_scholes_callprice,S_T,simulatedprice,Measure))

    sum_price=sum(simulatedprices)
    sum_finalstockprice=sum(final_portfolio_prices_St)
    
    squared_diffs = [(x - (sum_price/len(simulatedprices)) )** 2 for x in simulatedprices]
    varianceofprices = sum(squared_diffs) / len(simulatedprices)
    std_dev_simprices = math.sqrt(varianceofprices)
    standard_deviation_of_the_simprices.append(std_dev_simprices)
    avgsimulatedprice=np.sum(simulatedprices)/(len(simulatedprices))
    avgsimulatedprices.append(avgsimulatedprice) 
    
    squared_diff_finalstockprice = [(y - (sum_finalstockprice/len(final_portfolio_prices_St)) )** 2 for y in final_portfolio_prices_St]
    varianceofprices_finalstockprice = sum(squared_diff_finalstockprice) / len(final_portfolio_prices_St)
    std_dev_finalstockprice = math.sqrt(varianceofprices_finalstockprice)
    standard_deviation_of_the_final_portfolio_prices_St.append(std_dev_finalstockprice)
    
    avgS_T=np.sum(final_portfolio_prices_St)/(len(final_portfolio_prices_St))
    avgfinal_portfolio_prices_St.append(avgS_T) 
    
    squared_diffs_measure = [(x - sum(measure_values)/(len(measure_values)) )** 2 for x in measure_values]
    varianceofprices_measure = sum(squared_diffs_measure) / len(measure_values)
    std_dev_measure = math.sqrt(varianceofprices_measure)
        
    discounted_P_measure_value=np.sum(measure_values)/(len(measure_values))
    discounted_P_measure_values.append(discounted_P_measure_value)   
    
    Avg_unhedged_value=np.sum(unhedged_values)/(len(unhedged_values))
    Avg_unhedged_values.append(Avg_unhedged_value) 
    
    trading_just_asset1.append((alpha,black_scholes_callprice,avgsimulatedprice,std_dev_simprices,avgS_T,std_dev_finalstockprice,discounted_P_measure_value,std_dev_measure,Avg_unhedged_value))

df = pd.DataFrame(resultsTC_z2_i_hedging_asset_one_in_presence_of_costs, columns=['alpha', 'black_scholes_callprice', 'S_T', 'simulatedprice','Measure'])

df['alpha'] = df['alpha'].round(1)
df['black_scholes_callprice'] = df['black_scholes_callprice'].round(2)
df['S_T'] = df['S_T'].round(2)
df['simulatedprice'] = df['simulatedprice'].round(2)
df['Measure']=df['Measure'].round(2)

df.to_csv('resultsTC_z2_i_hedging_asset_one_in_presence_of_costs.csv', index=False, float_format='%.2f')
"""
print("Results have been written to resultsTC_z2_i_hedging_asset_one_in_presence_of_costs.csv")

# Just to view the contents in the csv file from here
with open('resultsTC_z2_i_hedging_asset_one_in_presence_of_costs.csv', 'r') as f:
    rows = [line.strip().split(',') for line in f]

# Create a table from the rows
table_output = table(rows)

# Print the table
print(table_output)
"""
df1 = pd.DataFrame(trading_just_asset1, columns=['alpha','black_scholes_price', 'avgsimulatedprice', 'std_dev_simprices','avgS_T','std_dev_finalstockprice','discounted_P_measure_value','std_dev_measure','Avg_unhedged_value'])

df1['alpha'] = df1['alpha'].round(2)
df1['black_scholes_price']=df1['black_scholes_price'].round(2)
df1['avgsimulatedprice'] = df1['avgsimulatedprice'].round(2)
df1['std_dev_simprices'] = df1['std_dev_simprices'].round(2)
df1['avgS_T'] = df1['avgS_T'].round(2)
df1['std_dev_finalstockprice'] = df1['std_dev_finalstockprice'].round(2)
df1['discounted_P_measure_value'] = df1['discounted_P_measure_value'].round(2)
df1['std_dev_measure'] = df1['std_dev_measure'].round(2)
df1['Avg_unhedged_value'] = df1['Avg_unhedged_value'].round(2)

df1.to_csv('trading_just_asset1.csv', index=False, float_format='%.2f')
"""
print("Results have been written to trading_just_asset1.csv")
"""
with open('trading_just_asset1.csv', 'r') as f:
    rows = [line.strip().split(',') for line in f]
new_headers = ["alpha", "Black S price", "avgsim.price", "stdsimprice", "avgS_T", "std_S_T", "discounted_P_val","std_measure" "Avg_unhedged_val"]
def make_first_letter_bold(text):
    return f"\033[1m{text[0]}\033[0m{text[1:]}"

bold_headers = [make_first_letter_bold(header) for header in new_headers]

fixed_width = 10.5 

formatted_header = ' '.join(f"{header:<{fixed_width}}" for header in bold_headers)

formatted_rows = [formatted_header]  
for row in rows[1:]:  
    formatted_row = ' '.join(f"{cell:<{fixed_width}}" for cell in row)
    formatted_rows.append(formatted_row)

table_output = '\n'.join(formatted_rows)

print(table_output)

df = pd.read_csv('trading_just_asset1.csv') 
plt.figure(figsize=(10, 6))
plt.plot(df['alpha'], df['black_scholes_price'],'-o', label='BS price')
plt.plot(df['alpha'], df['avgsimulatedprice'],'-.', label='avg sim price')
plt.plot(df['alpha'], df['discounted_P_measure_value'],'-*', label='dis P_measure')
plt.plot(df['alpha'], df['Avg_unhedged_value'], label='Avg_unhedged')

plt.xlabel('alpha values',fontsize =14)
plt.ylabel('Simulated Values',fontsize =14)
plt.legend()

plt.show()
